Test for using actionable-recourse, provided on https://github.com/ustunb/actionable-recourse

In order to compare recourse for several similar classifiers, we use cross validation to fit several logistic regression models (Is this the right way?). In the next step, we want to check whether the flipsets generated for one of them apply also for the other classifiers.

In [1]:
import copy
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold as CVGenerator
from sklearn.model_selection import GridSearchCV
import recourse as rs
from recourse.builder import ActionSet #FIX
from recourse.flipset import Flipset #FIX
from recourse.auditor import RecourseAuditor #FIX

import data

In [27]:
url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
df = pd.read_csv(url, skipinitialspace=True)
y, X = df.iloc[:, 0], df.iloc[:, 1:]

NEW: Use Cross validation to train several different classifiers

In [44]:
alter_C = False
n_splits = 20
if not alter_C:
    clf = LogisticRegression(max_iter=10000)
    cv = cross_validate(clf, X, y, cv=n_splits, return_estimator=True)
    cv_scores = cv['test_score']
    classifiers = np.array(cv['estimator'])

Alternative: Use GridSearchCV on parameter C
**TODO:** NameError: name 'tqdm_notebook' is not defined

In [46]:
if alter_C:
    cv_generator = CVGenerator(n_splits = 10, random_state = 42)

    # this code is for general purpose train/test evaluation using GridSearchCV
    gridsearch = GridSearchCV(
        clf, param_grid={"C":[1.0 / np.exp(l) for l in np.linspace(0, 3, num=n_splits)]},
        scoring='neg_mean_squared_error',
        return_train_score=True,
        cv=cv_generator,
        verbose=1,
        n_jobs=-1
    )

    gridsearch.fit(X,y)
    grid_search_df = pd.DataFrame(gridsearch.cv_results_)

    # cache a model for each parameter combination, trained on all data
    model_dict = {}
    classifiers = []
    grid_search_df['key'] = pd.np.nan
    for idx, p in tqdm_notebook(list(grid_search_df.params.iteritems())):
        model = copy(clf.set_params(**p)).fit(X,y)

        key = '__'.join(map(lambda x: '%s_%s' % x, p.items()))
        model_dict[key] = model
        grid_search_df.loc[idx, 'key'] = key
        classifiers.append(model)
# MAYDO: To actually use this variant, the next part would have to be adapted
    grid_search_df

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  9.9min finished
/home/valerie/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


NameError: name 'tqdm_notebook' is not defined

NEW: Select those classifiers that achieve performance within certain tolerance

In [41]:
#X_test = X[:3]
#for i, est in enumerate(scores['estimator']):
#    print(scores['test_score'][i], est.predict(X_test))
tolerance = 2*np.std(cv_scores)
good_classifiers = classifiers[cv_scores >= np.max(cv_scores) - tolerance]

print(np.max(cv_scores))
print(cv_scores.std())
print(len(good_classifiers))

0.826
0.009025580929287106
9


get predictions

In [5]:
yhat = [clf.predict(X) for clf in good_classifiers]

customize the set of actions and align

In [6]:
action_sets=[]
for clf in good_classifiers:
    ## matrix of features. ActionSet will learn default bounds and step-size.
    A = ActionSet(X)
    ## specify immutable variables
    A['Married'].mutable = False 
    ## can only specify properties for multiple variables using a list
    A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].mutable = False 
    A['EducationLevel'].step_direction = 1  ## force conditional immutability.
    A['EducationLevel'].step_size = 1  ## set step-size to a custom value.
    A['EducationLevel'].step_type = "absolute"  ## force conditional immutability.
    A['EducationLevel'].bounds = (0, 3)
    A['TotalMonthsOverdue'].step_size = 1  ## set step-size to a custom value.
    A['TotalMonthsOverdue'].step_type = "absolute"  ## discretize on absolute values of feature rather than percentile values
    A['TotalMonthsOverdue'].bounds = (0, 100)  ## set bounds to a custom value.
    
    ## tells `ActionSet` which directions each feature should move in to produce positive change.
    A.align(clf)
    action_sets.append(A)

Not necessary: testing if action sets were aligned correctly

In [7]:
for j,clf in enumerate(good_classifiers):
    for i,c in enumerate(X.columns):
        if action_sets[j]._elements[c].flip_direction != np.sign(clf.coef_[:,i]):
            print("Not well aligned", j, i)

**TODO:** We should get all flipsets and test them. But maybe first get it right for one...

In [8]:
j_clf = 0
i = np.flatnonzero(yhat[j_clf] <= 0)[0]

build a flipset for one individual

In [18]:
%%capture 
#discard the output of this cell (fs.populate prints way to much)
fs = Flipset(x = [X.iloc[i]], action_set = action_sets[j_clf], clf = good_classifiers[j_clf])
fs.populate(enumeration_type = 'distinct_subsets', total_items = 10)

**TODO:** How can we test whether the Flipset actually changes the outcome?

apply a flipset, **TODO**: Test the outcome given several classifiers

**CONCERN:** why does fs._df['x'][j] not match x[f]? (Also according to the code, this should be the old value of the features)

In [34]:
print(X.iloc[i])
x = X.iloc[i].copy()
k = 1
# apply the j-th action in the flipset 
for j, f in enumerate(fs._df['features'][k]):
    print("set", f, "from", x[f], "to", fs._df['x_new'][k][j])
    x[f] = fs._df['x_new'][k][j]
print(x)
fs._df
#def apply_flipset(fs, x):
    

Married                                    1
Single                                     0
Age_lt_25                                  1
Age_in_25_to_40                            0
Age_in_40_to_59                            0
Age_geq_60                                 0
EducationLevel                             2
MaxBillAmountOverLast6Months             120
MaxPaymentAmountOverLast6Months           20
MonthsWithZeroBalanceOverLast6Months       0
MonthsWithLowSpendingOverLast6Months       6
MonthsWithHighSpendingOverLast6Months      0
MostRecentBillAmount                     120
MostRecentPaymentAmount                    0
TotalOverdueCounts                         1
TotalMonthsOverdue                         4
HistoryOfOverduePayments                   1
Name: 0, dtype: int64
set MaxPaymentAmountOverLast6Months from 20 to 110.0
set TotalMonthsOverdue from 4 to 3.0
Married                                    1
Single                                     0
Age_lt_25                        

,cost,size,features,feature_idx,x,x_new,score_new,yhat_new,feasible,flipped
0,0.790661,1,[TotalMonthsOverdue],[15],[7.0],[3.0],0.045469,1.0,True,True
1,0.819801,2,"[MaxPaymentAmountOverLast6Months, TotalMonthsO...","[8, 15]","[100.0, 7.0]","[110.0, 3.0]",0.045779,1.0,True,True
2,0.829566,2,"[MostRecentBillAmount, TotalMonthsOverdue]","[12, 15]","[2010.0, 7.0]","[1926.0, 3.0]",0.054962,1.0,True,True
3,0.831567,2,"[MostRecentPaymentAmount, TotalMonthsOverdue]","[13, 15]","[100.0, 7.0]","[105.0, 3.0]",0.047410,1.0,True,True
4,0.838015,2,"[MaxBillAmountOverLast6Months, TotalMonthsOver...","[7, 15]","[2060.0, 7.0]","[2166.0, 3.0]",0.058650,1.0,True,True
5,0.840334,3,"[MaxBillAmountOverLast6Months, MostRecentBillA...","[7, 12, 15]","[2060.0, 2010.0, 7.0]","[2622.0, 1926.0, 4.0]",0.007364,1.0,True,True
6,0.858707,3,"[MaxPaymentAmountOverLast6Months, MostRecentBi...","[8, 12, 15]","[100.0, 2010.0, 7.0]","[110.0, 1926.0, 3.0]",0.055273,1.0,True,True
7,0.860707,3,"[MaxPaymentAmountOverLast6Months, MostRecentPa...","[8, 13, 15]","[100.0, 100.0, 7.0]","[110.0, 105.0, 3.0]",0.047721,1.0,True,True
8,0.867155,3,"[MaxBillAmountOverLast6Months, MaxPaymentAmoun...","[7, 8, 15]","[2060.0, 100.0, 7.0]","[2166.0, 110.0, 3.0]",0.058961,1.0,True,True
9,0.869474,4,"[MaxBillAmountOverLast6Months, MaxPaymentAmoun...","[7, 8, 12, 15]","[2060.0, 100.0, 2010.0, 7.0]","[2622.0, 110.0, 1926.0, 4.0]",0.007675,1.0,True,True


In [35]:
from IPython.display import HTML
HTML(fs.to_html())

**TODO:** Adapt from here to end: (How) do we want t use the auditor?

Run Recourse Audit on Training Data

In [ ]:
auditor = RecourseAuditor(action_sets[j_clf], coefficients = good_classifiers[j_clf].coef_[0], intercept = good_classifiers[j_clf].intercept_[0])
audit_df = auditor.audit(X)  ## matrix of features over which we will perform the audit.

print mean feasibility and cost of recourse

In [13]:
print(audit_df['feasible'].mean())
print(audit_df['cost'].mean())

1.0
0.044684389267020806


**TODO**: Generate flipsets for each good classifier and test whether it changes the outcame when the ather good classifiers are used.